# Arrays, cellular automata, and percolation 
## (Answers by David Miles Testa, dmt2943)

Imports: Make sure these are installed in your conda env, and that the conda env is active in your notebook
+ numpy
+ matplotlib
+ scipy
+ jupyter

For interactive plots:

+ ipykernel

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


%load_ext autoreload
%autoreload 2

: 

# The Abelian sandpile

### Background

We are going to implement the celebrated [Bak-Tang-Wiesenfeld model](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.59.381), also known as the "Abelian sandpile." This cellular automaton describes a lattice in which sand grains are continuously dropped onto random locations at a fixed rate, resulting in the formation of random sandpiles. When the sandpiles grow too high, they topple, resulting in avalanches that distribute grains to all of their neighbors.

If we denote the number of grains at a site $(x, y)$ as $z(x, y)$, a single "topple" event of the BTW model has the following update rule, which triggers only when $z \geq 4$.

$$
z(x, y) \rightarrow z(x, y) - 4
$$

$$
\begin{array}{lr}
    z(x + 1, y) \leftarrow z(x + 1, y) + 1    \\
    z(x - 1, y) \leftarrow z(x - 1, y) + 1    \\
    z(x, y + 1) \leftarrow z(x, y + 1) + 1    \\
    z(x, y - 1) \leftarrow z(x, y - 1) + 1    \\
\end{array}
$$

A topple event thus consists of a single site decreasing in height by four grains, and then distributing these grains to each of its north, south, east, and west neighbors---but not its diagonal neighbors.

After a single topple event, the entire lattice is checked again to see if any other sites now have $z \geq 4$, in which case those sites are toppled, too. If a site located at the boundary of the domain topples, then any grains that would go out of bounds are assumed to be permanently lost from the system. After all sites reach a state where $z \leq 4$, the avalanche has concluded. We then add a sand grain to a random new site, and calculate any new resulting avalanches.

This idealized system has several interesting properties: the continuous addition of grains represents a slow-timescale driving process, which effectively injects energy into the system. The avalances represent fast-timescale response dynamics, and the grains that fall off the edges represent dissipation---they prevent avalanches from continuing forever undriven. Despite its seeming simplicity, the BTK model represents perhaps the earliest widely-studied toy model of "self-organized criticality," a well-known hypothesis in nonequilibrium statistical physics that driven, dissipative systems tend to tune themselves into maximally-critical states (there are always sandpiles on the verge of toppling). The sandpile has been used as a thought experiment describing an incredible array of diverse systems, including: starts and stops in dragging friction, [earthquakes](https://agupubs.onlinelibrary.wiley.com/doi/abs/10.1029/JB094iB11p15635), timings between geyser eruptions, [timings of pulsar glitches](https://arxiv.org/abs/1403.6528), [fluctuations in an ultracold atomic gas](https://www.nature.com/articles/s41586-019-1908-6), [neuronal activity patterns in the brain](https://www.frontiersin.org/articles/10.3389/fnsys.2014.00166/full), [flux pinning](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.67.919), and many other areas.

A simple signature of self-organized-criticality observed in the sandpile model, which also represents an easy readout for experimental data, is the appearance of $1/f$ noise in the system's power spectrum, where $f$ denotes frequency. This distribution implies that avalanche durations have a skewed distribution, where larger events happen less frequently. For an experimental system, this results in "crackling" or brown noise in the detector. In the BTW model, the authors note that a $1/f$ distribution of rare events in the frequency corresponds to a $1/T$ distribution of waiting times between rare events. 

Here we will implement the BTW model, and then test for the appearance of $1/f$ noise.

## To Do:

1. Implement the Abelian Sandpile model and simulate its dynamics. I've included a template of the iterative solution below, although feel free to re-factor if you would prefer to implement the problem a different way. To my knowledge, there are three different ways to implement the Abelian sandpile:
    + The iterative solution involves adding a grain and then repeatedly checking the lattice for piles to topple
    + The recursive depth-first-search solution adds a grain and then traces the avalanche that results from each grain toppled by the initial addition.
    + The breadth-first-search solution adds a grain and then simultaneously tracks the four potential avalanches that result from that single grain.
2. Using the code included below, show that avalanche durations exhibit a $1/T^\alpha$ distribution, where $\alpha$ is some constant.
3. If a single image of the sandpile has size $N$ (the total number of lattice sites), what do you expect to be the worst-case scaling of the runtime of your algorithm? What about the expected scaling of memory usage?
4. You may have noticed that the waiting time and avalanche size distribution exhibit anomalous scaling in their tails, as visible as a small second peak near the extreme end of the distribution. What causes this effect?
5. (Hard, optional) Try [vectorizing](https://realpython.com/numpy-array-programming/) your sandpile implementation, in order to reduce the number of "for" loops used in your implementation. How does vectorization affect the runtime?

**If any experienced student wants to try to implement the Kramer & Marder paper discussed below instead of doing the BTK model, feel free to attempt it. I don't have a solution for it, but I would love to see one.**

## My Answers to the Sandpiles Questions:

1. I implemented a method for the cascade/avalanche that (from your description above) seems to be a recursive depth-first method. It reproduces the same $\frac{1}{T^{\alpha}}$ distributions that Prof. Gilpin's code creates.

2. I added some code below in the plotting block that fits a curve of the form $\frac{B}{T^{\alpha}}$ to the avalanche duration data, where B is a normalisation and $\alpha$ is the scaling. The code calculates a fit of $\alpha = 1.77536162$ with a corresponding covariance matric entry of $7.71699480\times 10^{-3}$ which implies a good fit. We can therefore conclude that a $T^{-\alpha}$ scaling is appropriate.

3. Since the number of grid points scales like $N\times N$, I would expect my algorithm to show a $N^2$ runtime scaling (as worst comes to worst we have to search the whole grid), but with a low coefficient out front due to it being a breadth first search through the grid. I added some code below to verify this experimentally, which it does. Since I store the whole grid, I would also expect my algorithm to have $N^2$ memory usage scaling. 

4. This is just a guess, but I assume it has something to do with the boundary conditions? The extra bump at the end of the waiting time distribution seems to me like the grid settles down after a while, with the sand being evenly distributed. It would then take longer for any single grain to start a cascade, at which point the whole grid would topple. I see this as there being a typical timescale where grid has smaller avalanches that spread the sand around evenly, and then a long timescale where the evenly spread sand reaches 4 grains almost everywhere, and then topples all at once. 

5. I tried vectorising (see the code below), and for the life of me I can't work out why it doesn't reproduce the same results as my recursive code. It almost exactly reproduces the $\alpha$ coefficient, but not the exact same final state. My vectorisation basically adds zeros around the rim of the grid so that I can create a stencil to add grains to the gridpoints that need to topple (see the code for more detail). Vectorisation in this case would just reduce the coefficient out front of the $N^2$ runtime scaling. 
  

In [2]:
class AbelianSandpile: 
    """
    An Abelian sandpile model simulation. The sandpile is initialized with a random
    number of grains at each lattice site. Then, a single grain is dropped at a random
    location. The sandpile is then allowed to evolve until it is stable. This process
    is repeated n_step times.

    A single step of the simulation consists of two stages: a random sand grain is 
    dropped onto the lattice at a random location. Then, a set of avalanches occurs
    causing sandgrains to get redistributed to their neighboring locations.
    
    Parameters:
    n (int): The size of the grid
    grid (np.ndarray): The grid of the sandpile
    history (list): A list of the sandpile grids at each timestep
    """

    def __init__(self, n=100, random_state=None):
        self.n = n
        np.random.seed(random_state) # Set the random seed
        self.grid = np.random.choice([0, 1, 2, 3], size=(n, n))
        self.history =[self.grid.copy()] # Why did we need to copy the grid?
        self.sites_to_check = [] #an array of sites that we need to loop through in the 
        #cascade. 


        
    def cascade(self):
        """
        This function checks whether the site at i_index, j_index needs to topple. If it does
        it topples the pile, sending grains to it's viable neighbours (obeying the absorbing BCs)
        and adding those neighbours to the list of sites to check (sites_to_check). It then removes
        the first element of the sites_to_check array, as that was the point that started the cascade. 
        
        """

        while len(self.sites_to_check) != 0: #as long as there are sites to check.
            for site in self.sites_to_check:
                if self.grid[site[0], site[1]] < 4:
                    self.sites_to_check = self.sites_to_check[1:] #cutting off the site
                else:
                    self.grid[site[0], site[1]] -= 4 #toppling.
                    #sending sandgrains to the surronding piles, but respecting BCs. 
                    if site[0] > 0:
                        self.grid[site[0]-1, site[1]] += 1
                        self.sites_to_check.append([site[0]-1, site[1]])
                    if site[0] < (self.n-1):
                        self.grid[site[0]+1, site[1]] += 1
                        self.sites_to_check.append([site[0]+1, site[1]])
                    if site[1] > 0:
                        self.grid[site[0], site[1]-1] += 1
                        self.sites_to_check.append([site[0], site[1]-1])
                    if site[1] < (self.n-1):
                        self.grid[site[0], site[1]+1] += 1
                        self.sites_to_check.append([site[0], site[1]+1])
                    self.sites_to_check = self.sites_to_check[1:]
                        
        

    def step(self):
        """
        Perform a single step of the sandpile model. Step corresponds a single sandgrain 
        addition and the consequent toppling it causes. 

        Returns: None
        """
        
        ########## YOUR CODE HERE ##########
        #
        #To get my code to reproduce Gilpin's distributions, I had to use the following
        #line from his solutions to get the exact same ICs. Apologies. I include below
        #how I initially did it, which works but produces a slightly different set of ICs. 
        new_sand_x, new_sand_y = np.random.choice(self.n, 2)
        
        
        #Adding a grain of sand somewhere on the grid at random. 
        #new_sand_x = np.random.randint(0, self.n - 1)
        #new_sand_y = np.random.randint(0, self.n - 1)
        
        self.grid[new_sand_x, new_sand_y] += 1
        self.sites_to_check.append([new_sand_x, new_sand_y])
        
        self.cascade()
        
        


        
        ########## YOUR CODE HERE ##########
        
        


    # we use this decorator for class methods that don't require any of the attributes 
    # stored in self. Notice how we don't pass self to the method
    @staticmethod
    def check_difference(grid1, grid2):
        """Check the total number of different sites between two grids"""
        return np.sum(grid1 != grid2)

    
    def simulate(self, n_step):
        """
        Simulate the sandpile model for n_step steps.
        """
        ########## YOUR CODE HERE ##########


        for n in range(0, n_step):
            self.step()
            if n%1000 == 0:
                print("made it to step: " + str(n))
            if self.check_difference(self.grid, self.history[-1]) > 0:
                self.history.append(self.grid.copy())
        
        return(self.grid)
        #
        #
        # YOUR CODE HERE. You should use the step method you wrote above.
        #
        #
        ########## YOUR CODE HERE ##########


### Test and use your code

+ You don't need to write any code below, these cells are just to confirm that everything is working and to play with your sandpile implementation
+ If you are working from a local fork of the entire course, then you already have access to the solutions. In this case, make sure to `git pull` to make sure that you are up-to-date (save your work first).
+ If you are working from a single downloaded notebook, or are working in Google Colab, then you will need to manually download the solutions file from the course repository. The lines below will do this for you.

In [ ]:
# Check if the "solutions" directory exists. If not, create it and download the solution file
import os
if not os.path.exists('solutions'):
    os.makedirs('solutions')
else:
    print('Directory "solutions" already exists. Skipping creation.')
!wget -P solutions https://raw.githubusercontent.com/williamgilpin/cphy/main/hw/solutions/sandpile.py

In [ ]:
## Import William's solution from answer key
# from solutions.sandpile import AbelianSandpile
# from solutions.sandpile import AbelianSandpileBFS as AbelianSandpile
# from solutions.sandpile import AbelianSandpileDFS as AbelianSandpile


# Run sandpile simulation
import scipy.optimize
def sandpile_fit(T, alpha, beta):
    """Just a function that I'll fit to later. 

    Args:
        T (int): The waiting time. 
        alpha (float): exponent in the timescale model. 
        beta (float): A normalisation factor. 
    """
    return(beta/(T**alpha))


model = AbelianSandpile(n=100, random_state=0)

initial_state = model.grid

plt.figure()
plt.imshow(model.grid, cmap='gray')
plt.title("Initial State")

model.simulate(10000)
plt.figure()
plt.imshow(model.grid, cmap='gray')
plt.title("Final state")




# Compute the pairwise difference between all observed snapshots. This command uses list
# comprehension, a zip generator, and argument unpacking in order to perform this task
# concisely.
all_events =  [model.check_difference(*states) for states in zip(model.history[:-1], model.history[1:])]
# remove transients before the self-organized critical state is reached
all_events = all_events[1000:]
# index each timestep by timepoint
all_events = list(enumerate(all_events))
# remove cases where an avalanche did not occur
all_avalanches = [x for x in all_events if x[1] > 1]
all_avalanche_times = [item[0] for item in all_avalanches]
all_avalanche_sizes = [item[1] for item in all_avalanches]
all_avalanche_durations = [event1 - event0 for event0, event1 in zip(all_avalanche_times[:-1], all_avalanche_times[1:])]


## Waiting time distribution
waiting_times = np.diff(np.array(all_avalanche_times))
plt.figure()
plt.semilogy()
plt.hist(waiting_times)
plt.title('Waiting Time distribution')
plt.xlabel('Waiting time')
plt.ylabel('Number of events')

## Duration distribution
log_bins = np.logspace(np.log10(2), np.log10(np.max(all_avalanche_durations)), 50) # logarithmic bins for histogram
vals, bins = np.histogram(all_avalanche_durations, bins=log_bins)



## Writing some code to fit T^(-\alpha) to the data
#first filtering out the zeros in the vals:
zeros_stencil = (vals != 0)
bins_minus_end = np.copy(bins[:-1])
vals_to_fit = vals[zeros_stencil]
bins_to_fit = bins_minus_end[zeros_stencil]


    
fit_coeffs, fit_covars = scipy.optimize.curve_fit(sandpile_fit, bins_to_fit, vals_to_fit, p0=[1.1, 1000])
fitted_vals = [sandpile_fit(T, fit_coeffs[0], fit_coeffs[1]) for T in bins_to_fit]


print(fit_coeffs)
print(fit_covars)


plt.figure()

plt.loglog(bins[:-1], vals, '.', markersize=10)
plt.loglog(bins_to_fit, fitted_vals, 'xr', markersize=10)
plt.title('Avalanche duration distribution')
plt.xlabel('Avalanche duration')
plt.ylabel('Count')

## Visualize activity of the avalanches
# Make an array storing all pairwise differences between the lattice at successive
# timepoints
all_diffs = np.abs(np.diff(np.array(model.history), axis=0))
all_diffs[all_diffs > 0] = 1
all_diffs = all_diffs[np.sum(all_diffs, axis=(1, 2)) > 1] # Filter to only keep big events
most_recent_events = np.sum(all_diffs[-100:], axis=0)
plt.figure(figsize=(5, 5))
plt.imshow(most_recent_events)
plt.title("Avalanche activity in most recent timesteps")


In [ ]:
#Setting up the experimental calculation of run-time scaling. This is ugly and dumb, but it works. 
import timeit

n_to_test = np.arange(1, 1000, 5)
time_taken = []
for n_points in n_to_test:
    time_taken.append(timeit.timeit("AbelianSandpile(n=n_points, random_state=0)", globals=globals(), number = 10))
    

plt.figure()
plt.plot(n_to_test, time_taken)
plt.show()


# My Attempt at Vectorisation
Below is my attempt at vectorising the sandpile simulation. For the life of me I don't undestand why it doesn't produce the exact same final state... 

In [6]:
class AbelianSandpileVectorised: 
    """
    An Abelian sandpile model simulation. The sandpile is initialized with a random
    number of grains at each lattice site. Then, a single grain is dropped at a random
    location. The sandpile is then allowed to evolve until it is stable. This process
    is repeated n_step times.

    A single step of the simulation consists of two stages: a random sand grain is 
    dropped onto the lattice at a random location. Then, a set of avalanches occurs
    causing sandgrains to get redistributed to their neighboring locations.
    
    Parameters:
    n (int): The size of the grid
    grid (np.ndarray): The grid of the sandpile
    history (list): A list of the sandpile grids at each timestep
    """

    def __init__(self, n=100, random_state=None):
        self.n = n
        np.random.seed(random_state) # Set the random seed
        self.grid = np.random.choice([0, 1, 2, 3], size=(n, n))
        self.history =[self.grid.copy()] # Why did we need to copy the grid?


    def step(self):
        """
        Perform a single step of the sandpile model. Step corresponds a single sandgrain 
        addition and the consequent toppling it causes. 

        Returns: None
        """
        
        ########## YOUR CODE HERE ##########
        #
        #
        # My solution starts by dropping a grain, and then solving for all topple events 
        # until the sandpile is stable. Watch your boundary conditions carefully.
        # We will use absorbing boundary conditions: excess sand grains fall off the edges
        # of the grid.
        
        #Adding a grain of sand somewhere on the grid at random. To get my code to reproduce Gilpin's distributions, 
        # I had to use the following line from his solutions to get the exact same ICs. Apologies. I initially just used
        #new_sand_x = np.random.randint(0, self.n - 1)
        #new_sand_y = np.random.randint(0, self.n - 1)
        #which works but produces a slightly different set of ICs. 
        
        new_sand_x, new_sand_y = np.random.choice(self.n, 2)
        self.grid[new_sand_y, new_sand_x] += 1
        
        #Now that we have added some sand to the grid, we have to check whether it caused any topples.


        piles_to_topple = (self.grid >= 4)
                
        while np.any(piles_to_topple) == True:
            #removing the piles that are 4 or taller. 
            self.grid[piles_to_topple] -= 4
            
            #we make a new array with an extra space around the rim. That way we can add sand to each 
            #space around the toppled piles, then slice away the edges of the grid when we are done. 
            expanded_grid = np.zeros((self.n+2, self.n+2))
            expanded_grid[1:-1, 1:-1] = self.grid
        
             
            #building the stencils to tell me where to add grains. 
            stencil_shifted_up = np.vstack((piles_to_topple, np.zeros((1, self.n), dtype = bool)))
            stencil_shifted_up = np.vstack((stencil_shifted_up, np.zeros((1, self.n), dtype = bool)))
            stencil_shifted_up = np.hstack((stencil_shifted_up, np.zeros((self.n + 2, 1), dtype = bool))) 
            stencil_shifted_up = np.hstack((np.zeros((self.n + 2, 1), dtype = bool), stencil_shifted_up))
             
            
            stencil_shifted_down = np.vstack((np.zeros((1, self.n), dtype = bool), piles_to_topple))
            stencil_shifted_down = np.vstack((np.zeros((1, self.n), dtype = bool), stencil_shifted_down))
            stencil_shifted_down = np.hstack((stencil_shifted_down, np.zeros((self.n + 2, 1), dtype = bool)))
            stencil_shifted_down = np.hstack((np.zeros((self.n + 2, 1), dtype = bool), stencil_shifted_down))
            
            
            stencil_shifted_left = np.hstack((piles_to_topple, np.zeros((self.n, 1), dtype = bool)))
            stencil_shifted_left = np.hstack((stencil_shifted_left, np.zeros((self.n, 1), dtype = bool)))
            stencil_shifted_left = np.vstack((stencil_shifted_left, np.zeros((1, self.n + 2), dtype = bool)))
            stencil_shifted_left = np.vstack((np.zeros((1, self.n + 2), dtype = bool), stencil_shifted_left))
            
            
            stencil_shifted_right = np.hstack((np.zeros((self.n, 1), dtype = bool), piles_to_topple))
            stencil_shifted_right = np.hstack((np.zeros((self.n, 1), dtype = bool), stencil_shifted_right))
            stencil_shifted_right = np.vstack((stencil_shifted_right, np.zeros((1, self.n + 2), dtype = bool)))
            stencil_shifted_right = np.vstack((np.zeros((1, self.n + 2), dtype = bool), stencil_shifted_right))
            
            expanded_grid[stencil_shifted_down] += 1
            expanded_grid[stencil_shifted_left] += 1
            expanded_grid[stencil_shifted_right] += 1
            expanded_grid[stencil_shifted_up] += 1
            
            self.grid = expanded_grid[1:-1, 1:-1]
            piles_to_topple = (self.grid >= 4) #checking to see if there are any more piles to topple. 
        
        ########## YOUR CODE HERE ##########
        
        


    # we use this decorator for class methods that don't require any of the attributes 
    # stored in self. Notice how we don't pass self to the method
    @staticmethod
    def check_difference(grid1, grid2):
        """Check the total number of different sites between two grids"""
        return np.sum(grid1 != grid2)

    
    def simulate(self, n_step):
        """
        Simulate the sandpile model for n_step steps.
        """
        ########## YOUR CODE HERE ##########


        for n in range(0, n_step):
            self.step()
            if n%1000 == 0:
                print("made it to step: " + str(n))
            if self.check_difference(self.grid, self.history[-1]) > 0:
                self.history.append(self.grid.copy())
        
        return(self.grid)
        #
        #
        # YOUR CODE HERE. You should use the step method you wrote above.
        #
        #
        ########## YOUR CODE HERE ##########


### Additional information and follow-up

+ Another member of our department, Michael Marder, has a [paper from 1992 in which he and his student Steve Kramer propose a cellular automaton model of river formation.](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.68.205) (Figure 2 is gorgeous!). The update rules and structure of implementing this simulation have many simularities to the BTK model, but, unlike the BTK model, there are not any open-source implementations of which I am aware.
+ Among cellular automata, the BTW model is not only unique for its asynchronous nature, but also because it has a "derivative" ruleset in contrast to the "integral"/totalistic ruleset we see in better-known systems like the Game of Life: instead of updating a cell based on the sum of all its neighbors, we update all neighbors based on the state of a cell.
+ The BTK model is called the "Abelian" sandpile because the final stable configuration is invariant to the order of toppling events. For example, instead of toppling the pile to completion after each addition, we could add $M$ several grains at once, and then perform a series of topples selecting unstable sites in any order until a stable configuration is achieved. The final sandpile would be identical to the one we would get if we simulated the pile through $M$ full additions sequentially.


### Extra  code for animations
+ No need to run this, but this code allows you to visualize the sandpile as it evolves over time.

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

activity_sliding2 = all_diffs[-500:]
vmin = np.percentile(activity_sliding2, 1)
# vmin = 0
vmax = np.percentile(activity_sliding2, 99.8)

# Assuming frames is a numpy array with shape (num_frames, height, width)
frames = np.array(activity_sliding2).copy() 

fig = plt.figure(figsize=(6, 6))
img = plt.imshow(frames[0], vmin=vmin, vmax=vmax);
plt.xticks([]); plt.yticks([])
# tight margins
plt.margins(0,0)
plt.gca().xaxis.set_major_locator(plt.NullLocator())

def update(frame):
    img.set_array(frame)

ani = FuncAnimation(fig, update, frames=frames, interval=50)
HTML(ani.to_jshtml())

In [ ]:

all_diffs = np.abs(np.diff(np.array(model.history), axis=0))
# all_diffs = all_diffs[np.sum(all_diffs, axis=(1, 2)) > 1] # Filter to only keep big events

# Use a trick to calculate the sliding cumulative sum
activity_cumulative = np.cumsum(all_diffs, axis=0)
# activity_sliding = activity_cumulative[50:] - activity_cumulative[:-50]
activity_sliding = all_diffs

plt.figure(figsize=(5, 5))
plt.imshow(activity_sliding[-1])


I don't understand why this cell doesn't run, but it just makes the movie? I am going to comment it out for now so the rest can go ahead and run.

In [ ]:
"""
# This code saves the sliding cumulative sum as a movie. No need to run this cell


activity_sliding2 = activity_sliding[-500:]
vmin = np.percentile(activity_sliding2, 1)
# vmin = 0
vmax = np.percentile(activity_sliding2, 99.8)
for i in range(len(activity_sliding2) - 1):
    
    out_path = "private_dump/sandpile/frame" + str(i).zfill(4) + ".png"

    plt.figure()
    plt.imshow(activity_sliding2[i], vmin=vmin, vmax=vmax)

    ax = plt.gca()
    ax.set_axis_off()
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    ax.set_aspect(1, adjustable='box')

    plt.savefig(out_path, bbox_inches='tight', pad_inches=0.0, dpi=300)
    plt.close() 
"""


# Percolation

The BTK cellular automaton represents a great example of a simple, toy computational model that exhibits non-trivial properties found in other, much more elaborate problems. It can be considered a [dynamical universality class](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.72.1690), meaning that many problems map onto it, independently of their microscopic details.

Here, we will consider a well-known example of a universality class describing a nonequilibrium phase transition: the directed percolation problem. This problem is usually attributed to [Broadbent & Hammersley in 1957](https://www.cambridge.org/core/journals/mathematical-proceedings-of-the-cambridge-philosophical-society/article/percolation-processes/C00CC4943F48228F8AC8031092FE84EC).

The basic idea is deceptively simple: given a D-dimensional lattice (for our purposes, a 2D grid), we randomly mark sites as "blocked" with probability $p$. The remaining sites are "open" for water to flow. For a given value of $p$, what is the probability that water poured into the top of the lattice will percolate to the bottom row through a chain of connected open sites? This problem is *directed percolation* because it has a preferred direction (water flows from top to bottom). We will assume two open sites are connected if one is north, south, east, or west of the other---but *not* diagonal. This corresponds to [von Neumann neighborhood rules](https://en.wikipedia.org/wiki/Von_Neumann_neighborhood).

A percolation simulation should take a binary array, and return True if it percolates, and False if it does not. There are many ways to implement a percolation algorithm, most of which involve simulating the addition of water to the grid. 
+ *Iterative Solution.* One option would be start with the top row, mark all open sites as "filled," and then pass to the next row and search for open sites connected to filled sites. Some addtional care is required, however, because water can pass through channels within a row until it reaches sites that are not directly below filled sites. One workaround would be to perform multiple iterative passes until the filled lattice stops changing. Another edge case is the case where water needs to pass through an uphill channel in order to make it to the bottom---this can be solved by passing over the lattice first from top to bottom, and then vice versa.
+ *Depth-first search (DFS).* Another simulation option would be a depth-first-search, where we start from each site on the top row and we search for any chain of North/Sout/East/West hops that leads to the other side of the lattice, marking all visited sites as filled with water. This last methods demonstrates the conceptual simularity between directed percolation and solving a maze puzzle; the only difference that the "blocked" sites in a maze are non-random. 

Here we are going to implemented a directed percolation model, and then perform experiments with it in order to determine how the percolation probability depends on the fraction of blocked sites $p$.

## To Do

1. Implement a two dimensional directed percolation model in Python. I've included my code outline below.
    + The iterative solution simulates water pouring into the top row, and then iterate over rows and sites. With this solution, there is some difficulty regarding how to handle the case where water can flow through a channel from right to left, even though we normally iterate from left to right. This can be solved by passing over each row twice. However, there is also a the case where water needs to pass uphill through a channel before it can proceed downhill. This can be solved by passing over the lattice from top to bottom, and then vice versa.
    + Can you think of a faster way of solving this problem? (Hint: we may not have seen recursion in class yet, but it may be helpful). If you want to try implementing your solution, you will need to add an additional private method `_flow_recursive(i, j)` that contains the recursive logic
2. Perform replicate simulations using the code I've provided below, and create a plot showing how the probability of percolation changes as a function of $p$ (the blockage probability). Beyond seeing the percolation to clogging transition, you'll notice that the variance in the outcome of your simulations behaves unexpectedly. What is going on here? (Hint: If you've studied the Ising model, you've seen something like this before)
3. Theoretically, does the transition point seen in our empirical results align with your intuition? Why does its value differ from $p=0.5$? (Hint: think about sites versus connections between sites).
4. How does the memory usage and runtime of your percolation model implement scale with the lattice size? You can answer this empirically or theoretically.
5. One way to sweep the control parameter $p$ would be to start in a limit where most of the sites are blocked, and then gradually open up individual sites one at a time until the lattice percolates. A video of a simulation where the lattice is gradually opened can be seen [here](https://vimeo.com/747772333). How do the different events in this gradually unblocked percolation simulation relate to the two timescales we saw in the sandpile problem?

# My Answers to the Percolation Questions. 

1. Using a very similar method to how I solved the sandpile simulation, I implemented what I think qualifies as a recursive solution to the percolation problem. I use
a function called "waterfall" to do a depth first search through the maze. Percolation just calles that waterfall function. I wound up not using the "poll_neighbours" function.

2. By "behaves unexpectedly" do you mean the variance has finite width? I have studied the Ising model a little bit, but I am having trouble seeing a complete connection here, as here the connected regions have no energetic reason to want to connect/disconnect. I almost want to say this is like the formation of Peierls droplets, but that requires that certain spin orientations have lower energy than others, which is where the analogy falls apart. 

    My best guess (but it really is a wild guess) is that there is a sharp phase transition between clogged and unclogged, but that before this point there is still the possibilty for the grid to be clogged, and after the transition there is still a chance for the grid to be unclogged. 

3. I would have thought the transition would be at $p = 0.5$, but it appears to happen a bit earlier than that at $p \sim 0.4$. For water to flow, you need two sites to be connected, so you actually need $p$ to be less than $0.5$ for there to be a better than even chance of a connection forming. Thus the transition would actually occur somewhere before $p = 0.5$.

4. As in the sandpile simulation, since we are storing an $N\times N$ grid, the memory scaling should go like $N^2$. As I am more or less implementing the same algorithm as in the sandpile problem, I expect the runtime complexity to also scale like $N^2$. I include some code below to experimentally verify this, which it does. 

5. The point at which the grid suddenly allows water to flow is similar to the transition out of the quiescent (when the sand isn't evenly distributed so we have localised avalanches more frequently) period in the sandpile simulation, to the catastrophic largescale collapse we saw as a bump in the sandpile waiting time distribution. 

In [10]:
class PercolationSimulation:
    """
    A simulation of a 2D directed percolation problem. Given a 2D lattice, blocked sites
    are denoted by 0s, and open sites are denoted by 1s. During a simulation, water is
    poured into the top of the grid, and allowed to percolate to the bottom. If water
    fills a lattice site, it is marked with a 2 in the grid. Water only reaches a site
    if it reaches an open site directly above, or to the immediate left or right 
    of an open site.

    I've included the API for my solution below. You can use this as a starting point, 
    or you can re-factor the code to your own style. Your final solution must have a 
    method called percolate that creates a random lattice and runs a percolation 
    simulation and
    1. returns True if the system percolates
    2. stores the original lattice in self.grid
    3. stores the water filled lattice in self.grid_filled

    + For simplicity, use the first dimension of the array as the percolation direction
    + For boundary conditions, assume that any site out of bounds is a 0 (blocked)
    + You should use numpy for this problem, although it is possible to use lists 



    Attributes:
        grid (np.array): the original lattice of blocked (0) and open (1) sites
        grid_filled (np.array): the lattice after water has been poured in
        n (int): number of rows and columns in the lattice
        p (float): probability of a site being blocked in the randomly-sampled lattice
            random_state (int): random seed for the random number generator
        random_state (int): random seed for numpy's random number generator. Used to 
            ensure reproducibility across random simulations. The default value of None
            will use the current state of the random number generator without resetting
            it.
    """

    def __init__(self, n=100, p=0.5, grid=None, random_state=None):
        """
        Initialize a PercolationSimulation object.

        Args:
            n (int): number of rows and columns in the lattice
            p (float): probability of a site being blocked in the randomly-sampled lattice
            random_state (int): random seed for numpy's random number generator. Used to
                ensure reproducibility across random simulations. The default value of None
                will use the current state of the random number generator without resetting
                it.
        """

        self.random_state = random_state # the random seed

        # Initialize a random grid if one is not provided. Otherwise, use the provided
        # grid.
        if grid is None:
            self.n = n
            self.p = p
            self.grid = np.zeros((n, n))
            self._initialize_grid()
        else:
            assert len(np.unique(np.ravel(grid))) <= 2, "Grid must only contain 0s and 1s"
            self.grid = grid.astype(int)
            # override numbers if grid is provided
            self.n = grid.shape[0]
            self.p = 1 - np.mean(grid)

        # The filled grid used in the percolation calculation. Initialize to the original
        # grid. We technically don't need to copy the original grid if we want to save
        # memory, but it makes the code easier to debug if this is a separate variable 
        # from self.grid.
        self.grid_filled = np.copy(self.grid)
        self.sites_to_check = []

    def _initialize_grid(self):
        """
        Sample a random lattice for the percolation simulation. This method should
        write new values to the self.grid and self.grid_filled attributes. Make sure
        to set the random seed inside this method.

        This is a helper function for the percolation algorithm, and so we denote it 
        with an underscore in order to indicate that it is not a public method (it is 
        used internally by the class, but end users should not call it). In other 
        languages like Java, private methods are not accessible outside the class, but
        in Python, they are accessible but external usage is discouraged by convention.

        Private methods are useful for functions that are necessary to support the 
        public methods (here, our percolate() method), but which we expect we might need
        to alter in the future. If we released our code as a library, others might 
        build software that accesses percolate(), and so we should not alter the 
        input/outputs because it's a public method
        """
        ###############################################################################
        
        #
        ####### YOUR CODE HERE  ####### 

        """
        np.random.seed(self.random_state)
        self.grid = np.random.choice([0, 1], size=(self.n, self.n), p=[self.p, 1-self.p])
        self.grid_filled = np.copy(self.grid)
        """
        
        #Gilpin's code for the initialisation, necessary to get the same ICs.
        np.random.seed(self.random_state)
        self.grid = np.random.choice([1, 0], size=(self.n, self.n), p=[1-self.p, self.p])
        self.grid_filled = np.copy(self.grid)
        
        # Hint: my solution is 3 lines of code in numpy
        #
        #
        ###############################################################################
        

    def _flow_recursive(self, i, j):
        """
        Only used if we opt for a recursive solution.

        The recursive portion of the flow simulation. Notice how grid and grid_filled
        are used to keep track of the global state, even as our recursive calls nest
        deeper and deeper
        """
        
        ####### YOUR CODE HERE  #######################################################
        #
        # Remember to check the von Neumann neighborhood of the current site. There should
        # be 4 recursive calls in total, and 4 base cases
        #
        #
        ###############################################################################s


    def _poll_neighbors(self, i, j):
        """
        Check whether there is a filled site adjacent to a site at coordinates i, j in 
        self.grid_filled. Respects boundary conditions.
        """

        ####### YOUR CODE HERE  #######################################################
        # Hint: my solution is 4 lines of code in numpy, but you may get different 
        # results depending on how you enforce the boundary conditions in your solution.
        # Not needed for the recursive solution
        #
        #
        ###############################################################################



    def _flow(self):
        """
        Run a percolation simulation using recursion

        This method writes to the grid and grid_filled attributes, but it does not
        return anything. In other languages like Java or C, this method would return
        void
        """
        ###############################################################################
        raise NotImplementedError("Implement this method")
        ####### YOUR CODE HERE  ####### 
        # Hint: my non-recursive solution contains one row-wise for loop, which contains 
        # several loops over individual lattice sites. You might need to visit each lattice 
        # site more than once per row. In my implementation, split the logic of checking
        # the von neumann neighborhood into a separate method _poll_neighbors, which
        # returns a boolean indicating whether a neighbor is filled
        #
        # My recursive solution calls a second function, _flow_recursive, which takes 
        # two lattice indices as arguments

        ###############################################################################

    def waterfall(self):
        """
        This function checks whether the site at i_index, j_index has open (no water and not blocked) neighbours, 
        and is open itself. If it does, and is full of water itself, it sends water to it's viable neighbours, and 
        adds those neighbours to the list of sites to check (sites_to_check). It then removes itself
        (the first element of the sites_to_check array). 
        
        """
        while len(self.sites_to_check) != 0: #as long as there are sites to check.
            for site in self.sites_to_check:
                if site[1] < self.n-1:
                    if (self.grid_filled[site[0], site[1] + 1] == 1):
                        self.grid_filled[site[0], site[1] + 1] = 2
                        self.sites_to_check.append([site[0], site[1] + 1])
                if site[1] > 0:
                    if (self.grid_filled[site[0], site[1] - 1] == 1):
                        self.grid_filled[site[0], site[1] - 1] = 2
                        self.sites_to_check.append([site[0], site[1] - 1])
                if site[0] < self.n-1:
                    if (self.grid_filled[site[0] + 1, site[1]] == 1):
                        self.grid_filled[site[0] + 1, site[1]] = 2
                        self.sites_to_check.append([site[0] + 1, site[1]])
                if site[0] > 0:
                    if (self.grid_filled[site[0] - 1, site[1]] == 1):
                        self.grid_filled[site[0] - 1, site[1]] = 2
                        self.sites_to_check.append([site[0] - 1, site[1]])
                self.sites_to_check = self.sites_to_check[1:] #we've checked the neighbours, so we can remove the grid point.


    def percolate(self):
        """
        Initialize a random lattice and then run a percolation simulation. Report results
        """
        ###############################################################################

        ####### YOUR CODE HERE  #######     
        
        
        #I start off by adding zeros around the edges of the array so that I don't have to 
        #manually account for the boundary conditions with a whole mess of if statements. 
        self._initialize_grid()
        
        #doing the first row scan to make sure we don't have a capped off sponge, which would crash my _poll_neighbours
        #function. I'm sure you could vectorise this, but I have covid and can't think too well at the moment. 
        
        for j in range(0, self.n):
            if self.grid_filled[0, j] == 1:
                self.grid_filled[0, j] = 2
                self.sites_to_check.append([0, j])


        self.waterfall()
        
        if np.any(self.grid_filled[-1, :] == 2):
            return(True)
        else:
            return(False)      
        
        
        
        # Hint: my solution is 3 lines of code, and it just calls other methods in the
        # class, which do the heavy lifting

        ###############################################################################




### Test and use your code

+ You don't need to write any new code below, these cells are just to confirm that everything is working and to play with the your percolation implementation
+ If you are working from a local fork of the entire course, then you already have access to the solutions. In this case, make sure to `git pull` to make sure that you are up-to-date (save your work first).
+ If you are working from a single downloaded notebook, or are working in Google Colab, then you will need to manually download the solutions file from the course repository. The lines below will do this for you.

In [ ]:
# Check if the "solutions" directory exists. If not, create it and download the solution file
import os
if not os.path.exists('solutions'):
    os.makedirs('solutions')
else:
    print('Directory "solutions" already exists. Skipping creation.')
!wget -P solutions https://raw.githubusercontent.com/williamgilpin/cphy/main/hw/solutions/percolation.py

In [ ]:
# Import William's solution
# from solutions.percolation import PercolationSimulation
# from solutions.percolation_iterative import PercolationSimulation

from matplotlib.colors import LinearSegmentedColormap
def plot_percolation(mat):
    """
    Plots a percolation matrix, where 0 indicates a blocked site, 1 indicates an empty 
    site, and 2 indicates a filled site
    """
    cvals  = [0, 1, 2]
    colors = [(0, 0, 0), (0.4, 0.4, 0.4), (0.372549, 0.596078, 1)]

    norm = plt.Normalize(min(cvals), max(cvals))
    tuples = list(zip(map(norm,cvals), colors))
    cmap = LinearSegmentedColormap.from_list("", tuples)
    plt.imshow(mat, cmap=cmap, vmin=0, vmax=2)
#20

model = PercolationSimulation(n=20, random_state=0, p=0.1)
print(model.percolate())
plt.figure()
plot_percolation(model.grid_filled)

model = PercolationSimulation(n=20, random_state=0, p=0.4)
print(model.percolate())
plt.figure()
plot_percolation(model.grid_filled)


model = PercolationSimulation(n=20, random_state=0, p=0.9)
print(model.percolate())
plt.figure()
plot_percolation(model.grid_filled)

Run replicate simulations across replicates with different bond occupation probabilities

The percolation probability represents an effective order parameter for this system, and
so we will attempt to calculate the percolation probability by performing many replicate
simulations at different values of the control parameter $p$.

In [ ]:

# Import William's solution
# from solutions.percolation import PercolationSimulation

pvals = np.linspace(0, 1, 25) # control parameter for percolation phase transition
n_reps = 200 # number of times to repeat the simulation for each p value

all_percolations = list()
for p in pvals:
    print("Running replicate simulations for p = {}".format(p), flush=True)
    all_replicates = list()
    for i in range(n_reps):
        # Initialize the model
        model = PercolationSimulation(30, p=p)
        all_replicates.append(model.percolate())
    all_percolations.append(all_replicates)

plt.figure()
plt.plot(pvals, np.mean(np.array(all_percolations), axis=1))
plt.xlabel('Average site occupation probability')
plt.ylabel('Percolation probability')

plt.figure()
plt.plot(pvals, np.std(np.array(all_percolations), axis=1))
plt.xlabel('Standard deviation in site occupation probability')
plt.ylabel('Percolation probability')

plt.show()


## Just from curiousity, plot the distribution of cluster sizes at the percolation threshold
## why does it appear to be bimodal?
all_cluster_sizes = list()
p_c = 0.407259
n_reps = 5000
for i in range(n_reps):
    model = PercolationSimulation(100, p=p_c)
    model.percolate()
    cluster_size = np.sum(model.grid_filled == 2)
    all_cluster_sizes.append(cluster_size)

    if i % 500 == 0:
        print("Finished simulation {}".format(i), flush=True)

all_cluster_sizes = np.array(all_cluster_sizes)

plt.figure()
plt.hist(all_cluster_sizes, 50);


# Experimentally verifying the $N^2$ runtime scaling.

In [ ]:
#Setting up the experimental calculation of run-time scaling. This is ugly and dumb, but it works. 
import timeit

n_to_test = np.arange(1, 500, 5)
time_taken = []
for n_points in n_to_test:
    time_taken.append(timeit.timeit("PercolationSimulation(n=n_points, random_state=0)", globals=globals(), number = 10))
    

plt.figure()
plt.plot(n_to_test, time_taken)
plt.show()

# Follow-up and more information

+ Directed percolation maps onto all sorts of interesting problems in physics and graph theory. There are literal applications of percolation to [flow in porous media](https://iopscience.iop.org/article/10.1088/0305-4470/14/5/012/meta), as well as [electrical conductance](https://journals.aps.org/prb/abstract/10.1103/PhysRevB.26.5293) and [light propagation in disordered materials](https://journals.aps.org/prl/abstract/10.1103/PhysRevLett.99.234503). A great example of universality is [recent work](https://www.nature.com/articles/nphys3548) showing similar scaling exponents describe both ecological collapse and pipe turbulence, which both map onto the Directed Percolation universality class. [Here's another recent paper](https://journals.plos.org/ploscompbiol/article?id=10.1371/journal.pcbi.1007508) that uses percolation theory to understand electrical signalling between neighboring cells in a biofilm. 
+ If you are interested in the mathematical theory of percolation, [Kim Christensen's notes on the subject](https://web.mit.edu/ceder/publications/Percolation.pdf) are extremely clear and interesting.


## Optional code and extras

Create a simulation where we iteratively fill out one lattice, in order to make a video of our percolation simulation

In [15]:
initial_lattice = np.zeros((50, 50))

# Decide the order in which sites become blocked
np.random.seed(0)
all_lattice_indices = np.array(
    [(i, j) for i in range(initial_lattice.shape[0]) for j in range(initial_lattice.shape[1])]
)
np.random.shuffle(all_lattice_indices)

# does percolate 
all_grids = list()
for inds in all_lattice_indices:
    
    initial_lattice[inds[0], inds[1]] = 1
    model = PercolationSimulation(grid=initial_lattice)
    model.percolate()

    if (model.p > 0.3) and (model.p < 0.7):
        all_grids.append(np.copy(model.grid_filled))

    

In [16]:
## Make an interactive video
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets

def plotter(i):
    fig = plt.figure(figsize=(10, 10))
    plot_percolation(all_grids[i])
    plt.show()



interact(
    plotter, 
    i=widgets.IntSlider(0, 0, len(all_grids) - 1, layout=Layout(width='500px'))
)

# Appendix and Extras

This is William's leftover code for making videos and figures; no need to run this

In [ ]:
from matplotlib.animation import FuncAnimation
from IPython.display import HTML

activity_sliding2 = activity_sliding[-500:]
vmin = np.percentile(activity_sliding2, 1)
# vmin = 0
vmax = np.percentile(activity_sliding2, 99.8)

# Assuming frames is a numpy array with shape (num_frames, height, width)
frames = np.array(activity_sliding2).copy() 

fig = plt.figure(figsize=(6, 6))
# img = plt.imshow(frames[0], vmin=vmin, vmax=vmax);
plt.xticks([]); plt.yticks([])
# tight margins
plt.margins(0,0)
plt.gca().xaxis.set_major_locator(plt.NullLocator())

def update(frame):
    plot_percolation(all_grids[::2])

ani = FuncAnimation(fig, update, frames=all_grids[::2], interval=50)
HTML(ani.to_jshtml())

In [ ]:
# (Optional) I used this code to export still images, and then make a video from them using
# the command-line tool ffmpeg


for i in range(len(all_grids[::2]) - 1):
    
    
    out_path = "private_dump/percolation/frame" + str(i).zfill(4) + ".png"

    plt.figure()
    plot_percolation(all_grids[::2][i])

    ax = plt.gca()
    ax.set_axis_off()
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    ax.set_aspect(1, adjustable='box')

    plt.savefig(out_path, bbox_inches='tight', pad_inches=0.0, dpi=160)
    plt.close()



In [ ]:
# (Optional) I used this code to stitch the images together into a video. The %%bash magic tells
# iPython to treat these lines as bash commands, rather than Python. I then use the 
# command-line tool `ffmpeg` to stitch the images together into a video.

%%bash
ffmpeg -r 60 -i private_dump/percolation/frame%04d.png -vf "scale=trunc(iw/2)*2:trunc(ih/2)*2" -vcodec libx264 -pix_fmt yuv420p private_dump/percolation/vid2.mov
